# Multi-Query Retriever
* 개떡같이 질문해도 찰떡같이 알아듣도록, 사용자 질문을 여러개의 유사 질문으로 재생성

일반적사례
Q: B 은행의 대출은 어때?
A: B 은행의 대출은 좋은 편입니다.

Multi-Query Retriever
Q: B 은행의 대출은 어때?
Q1: B 은행의 대출 금리는 어때?
Q2: B 은행의 대출 조건는 어때?
Q3: B 은행의 대출 후기는 어때?

In [1]:
# Build a sample vectorDB
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

In [3]:
# Load blog post
loader = WebBaseLoader("https://n.news.naver.com/mnews/article/003/0012317114?sid=105")
data = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=0)
splits = text_splitter.split_documents(data)

# VectorDB
model_name = "jhgan/ko-sbert-nli"
encode_kwargs = {'normalize_embeddings': True}
ko_embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    encode_kwargs=encode_kwargs
)

vectordb = Chroma.from_documents(documents=splits, embedding=ko_embedding)

In [9]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.chat_models import ChatOpenAI

question = "삼성전자 갤럭시 S24는 어떨 예정이야?"
llm = ChatOpenAI(temperature=0)

retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectordb.as_retriever(), 
    llm=llm,
)

/Users/heewungsong/anaconda3/envs/pyautogen/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [10]:
# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [11]:
unique_docs = retriever_from_llm.get_relevant_documents(query=question)
len(unique_docs)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
INFO:langchain.retrievers.multi_query:Generated queries: ['1. 삼성 갤럭시 S24 출시 일정은 어떻게 되나요?', '2. 갤럭시 S24의 삼성전자 출시 계획은 어떻게 되나요?', '3. 삼성전자가 준비 중인 갤럭시 S24에 대한 정보를 알고 싶어요.']


5

In [12]:
unique_docs

[Document(page_content="[서울=뉴시스] 삼성전자가 17일 오전 10시(현지시간, 한국 시간 18일 오전 3시) 미국 캘리포니아주 산호세(새너제이)에서 '삼성 갤럭시 언팩 2024'를 열고 갤럭시 S24를 공개한다. 사진은 포르투갈에서 유출된 갤럭시 S24 시리즈 포스터 추정 이미지 (사진=theonecid 엑스 캡처)  *재판매 및 DB 금지[서울=뉴시스]윤정민 기자 = 인공지능(AI) 서비스가 대거 탑재될 삼성전자 플래그십 스마트폰 '갤럭시 S24'가 18일 베일을 벗는다. 갤럭시 S23이 전작 대비 카메라, 디자인 등 대폭 개선됐다면, 이번 신작은", metadata={'language': 'ko', 'source': 'https://n.news.naver.com/mnews/article/003/0012317114?sid=105', 'title': "언팩 D-4, 세계 최초 AI폰 '갤S24' 이렇게 나온다"}),
 Document(page_content="[서울=뉴시스] 삼성전자가 17일 오전 10시(현지시간, 한국 시간 18일 오전 3시) 미국 캘리포니아주 산호세(새너제이)에서 열 '삼성 갤럭시 언팩 2024'의 주제는 '모바일 AI의 새로운 시대 개막'이다. 앞서 삼성전자가 AI를 스마트폰 차기작 특징으로 예고했던 만큼 어떤 AI 기능이 실릴지 관심이 쏠린다.삼성전자가 공식적으로 밝힌 AI 서비스는 실시간 통화 통역이다. 이미 SK텔레콤 '에이닷' 등 통화 통역을 지원하는 앱이 있다. 하지만 자체 AI가 탑재될 갤럭시 S24는 별도 앱을 설치하지 않아도 통역 통화를 이용할 수", metadata={'language': 'ko', 'source': 'https://n.news.naver.com/mnews/article/003/0012317114?sid=105', 'title': "언팩 D-4, 세계 최초 AI폰 '갤S24' 이렇게 나온다"}),
 Document(page_content="[서울=뉴시스] 8일 업계에 따르면 삼성전자는 

In [14]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature = 0)
qa = RetrievalQA.from_chain_type(
    llm = llm, 
    chain_type = "stuff",
    retriever = retriever_from_llm,
    return_source_documents = True
)

result = qa(question)

/Users/heewungsong/anaconda3/envs/pyautogen/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
INFO:langchain.retrievers.multi_query:Generated queries: ['1. 삼성 갤럭시 S24 출시 일정은 어떻게 되나요?', '2. 삼성 갤럭시 S24의 기대되는 특징은 무엇인가요?', '3. 삼성 갤럭시 S24의 스펙 및 가격에 대한 정보를 알고 싶어요.']


In [16]:
print(result['result'])

삼성전자 갤럭시 S24는 새로운 AI 서비스와 함께 출시될 예정이며, 카메라, 디자인 등이 대폭 개선될 것으로 예상됩니다. 또한, 티타늄 소재를 사용한 울트라 모델과 퀄컴 스냅드래곤8 3세대 또는 삼성 엑시노스 2400 칩이 탑재될 것으로 보입니다. 색상은 오닉스 블랙, 마블 그레이, 코발트 바이올렛, 앰버 옐로우 등이 예상되며, 출고가는 일반·플러스 모델은 동결될 것으로 예상되지만, 울트라 모델은 약 10만원 정도 인상될 것으로 전망됩니다.
